# Bài thực hành số 5.1

Chào mừng đến với bài thực hành số 5.1, đây sẽ là phần tiếp diễn của bài thực hành thứ 5 trên colab. Qua bài thực hành này, chúng tôi sẽ giúp bạn biết các áp dụng model vừa được train vào việc nhận dạng vật thể trong video hoặc qua camera laptop.

## Hướng dẫn

Dưới đây là hướng dẫn chi tiết các bước để bạn có thể hiểu được quá trình áp dụng DeepSORT và YOLO lên video.


### Cài đặt thư viện cần thiết

In [21]:
import numpy as np
import torch
import cv2
import math
import time
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

### Tải model nhận diện

Đáp án bài tập 1

In [ ]:
# force reload: tránh xung đột tham số khi load model mới
model = torch.hub.load('ultralytics/yolov5', 'custom', path= r"D:\Summer 2024\Capstone project\Lab\YOLOv5 Test\custom_yolov5_car_v2\weights\best.pt", force_reload = True)

### Test với hình ảnh



In [ ]:
img_path = [...]

In [ ]:
img = cv2.imread(img_path)

# Hiển thị hình ảnh sau khi đổi kích thước
cv2.imshow("test", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
test_detect = model(img_path)
test_detect.show()

### Áp dụng với video

Đáp án bài tập 2

In [ ]:
# Đọc video đầu vào
input_video_path = r"D:\Summer 2024\Capstone project\Lab\YOLOv5 Test\Test_video\test_car_2.mp4"  # Đường dẫn đến video đầu vào
output_video_path = r"D:\Summer 2024\Capstone project\Lab\YOLOv5 Test\Test_video\test_car_2_output.mp4"  # Đường dẫn đến video đầu ra

# Mở video đầu vào
cap = cv2.VideoCapture(input_video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Lấy thông số của video
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Tạo video writer để lưu video đầu ra
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec cho định dạng MP4
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Chuyển đổi khung hình từ BGR sang RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Dự đoán với YOLOv5
    results = model(rgb_frame)

    # Lấy kết quả dự đoán dưới dạng DataFrame
    df = results.pandas().xyxy[0]

    # Vẽ các hộp dự đoán lên khung hình
    for _, row in df.iterrows():
        x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
        label = row['name']
        score = row['confidence']
        
        # Vẽ hộp bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # Vẽ nhãn lớp và độ tin cậy
        cv2.putText(frame, f'{label} {score:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Ghi khung hình vào video đầu ra
    out.write(frame)

# Giải phóng tài nguyên
cap.release()
out.release()
cv2.destroyAllWindows()
